# Chutes and Ladders

I read an article by Joel Smith about probabilities related to the popular children's board game, Chutes and Ladders. He specifically looked at how many turns it takes to complete a game for a given number of players. To perform this analysis, Smith used Markov Chains and Minitab software. I am using Monte Carlo simulation and Python to see how closely I can get to his results.

Smith, J. (2016, April 01). What Are the Odds? Chutes and Ladders. Retrieved July 10, 2017, from http://blog.minitab.com/blog/fun-with-statistics/what-are-the-odds-chutes-and-ladders

In [1]:
import random
import pandas as pd

# Set my random number generator seed to reproduce results
random.seed(9655)

## Movement on the Board

I first need a function for how pieces move on the board. The player first spins to see how many spaces to move. Normally, the player just moves that many spaces, but if the player gets a chute or ladder, they get moved to a predetermined space. Finally, if the resulting space is past the final spot (100), then the player does not move. 

In [2]:
# Function for movement based on the starting position and what value the player gets from the spinner
def move(start, spin):
    new_pos = start + spin
    if new_pos == 1:
        final_pos = 38
    elif new_pos == 4:
        final_pos = 14
    elif new_pos == 9:
        final_pos = 31
    elif new_pos == 16:
        final_pos = 6
    elif new_pos == 21:
        final_pos = 42
    elif new_pos == 28:
        final_pos = 84
    elif new_pos == 36:
        final_pos = 44
    elif new_pos == 48:
        final_pos = 26
    elif new_pos == 49:
        final_pos = 11
    elif new_pos == 51:
        final_pos = 67
    elif new_pos == 56:
        final_pos = 53
    elif new_pos == 62:
        final_pos = 19
    elif new_pos == 64:
        final_pos = 60
    elif new_pos == 71:
        final_pos = 91
    elif new_pos == 80:
        final_pos = 100
    elif new_pos == 87:
        final_pos = 24
    elif new_pos == 93:
        final_pos = 73
    elif new_pos == 95:
        final_pos = 75
    elif new_pos == 98:
        final_pos = 78
    elif new_pos > 100:
        final_pos = start
    else:
        final_pos = new_pos
    return final_pos

## Simulating a Game

Next, I need a function to simulate a single game and return the number of turns required to complete it. The first step is to place all of the pieces at the starting spot. After that, I loop through turns until a player wins using random number generation. It is possible although unlikely for the game to keep going in an endless loop. To prevent this, I have capped the number of turns at 1,000.

In [3]:
# Function to simulate a single game and return the number of tunrs that it took to complete the game
# Players - number of players (2-4 recommended)
# Start - starting square (final square is always 100)
# Slices - number of slices on the spinner with equal probability
def game(players, start=0, slices=6):
    
    # Starting state at the starting square
    game_state = {}
    turn = 0
    exitflag = False
    for player in range(players):
        game_state[player] = start
    
    # Go through turns with a maximum number of 1000 turns to avoid an endless loop
    while True:
        turn = turn + 1
        if turn == 1000:
            break
        for player in range(players):
            game_state[player] = move(game_state[player], random.randrange(1,slices+1)) 
            if game_state[player] == 100:
                exitflag = True
        if exitflag:
            break       
    return turn

## Simulate Many Games

I want to compare the point estimates shown in the article for 1 to 4 players. To approximate the results with simulation, I will use 100,000 iterations. I do not expect the simulated results to match the theoretical exactly but it should be close. As the number of simulations increase, then the difference should decrease.

In [4]:
# Simulate many games
sims = 100000

for j in [1,2,3,4]:
    games = {}
    for i in range(sims):
        games[i] = game(j)
    df = pd.DataFrame.from_dict(games, orient='index')
    df.columns = ['Turns']
    print("For {} Player(s):".format(j))
    print("Minimum of {} turns to complete the game".format(df['Turns'].min()))
    print("With a probability of {}".format(len(df[df['Turns']==df['Turns'].min()])/sims))
    print("90% of player-games will finish before turn {}".format(round(df.quantile(.9).values[0],0)))
    print("95% of player-games will finish before turn {}".format(round(df.quantile(.95).values[0],0)))
    print("99% of player-games will finish before turn {}".format(round(df.quantile(.99).values[0],0)))
    print("Median of {} turns to complete the game".format(df['Turns'].median()))
    print("Mean of {} turns to complete the game".format(round(df['Turns'].mean(),1)))
    print("...")

For 1 Player(s):
Minimum of 7 turns to complete the game
With a probability of 0.00147
90% of player-games will finish before turn 73.0
95% of player-games will finish before turn 90.0
99% of player-games will finish before turn 128.0
Median of 33.0 turns to complete the game
Mean of 39.5 turns to complete the game
...
For 2 Player(s):
Minimum of 7 turns to complete the game
With a probability of 0.00321
90% of player-games will finish before turn 44.0
95% of player-games will finish before turn 53.0
99% of player-games will finish before turn 73.0
Median of 23.0 turns to complete the game
Mean of 26.5 turns to complete the game
...
For 3 Player(s):
Minimum of 7 turns to complete the game
With a probability of 0.0048
90% of player-games will finish before turn 35.0
95% of player-games will finish before turn 40.0
99% of player-games will finish before turn 53.0
Median of 20.0 turns to complete the game
Mean of 21.7 turns to complete the game
...
For 4 Player(s):
Minimum of 7 turns to c

## Conclusions

My results are close to those in the article. I was surprised that even with 100,000 simulations, the mean was not closer to the Markov Chain results. My next move should be learning how to replicate the Markov Chain results themselves. 

Like Smith says, there is really no in-game advantage to understanding the statistics only that adding more players speeds up the game. 